# First Project

## Section Three - Problem no.1: Design a recommendation system

### Adding Libraries

In [178]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

### Importing and checking the data

In [179]:
data = pd.read_csv('../Divar.csv')

C:\Users\Abtin Rahbari\AppData\Local\Temp\ipykernel_22264\2330768946.py:1: DtypeWarning: Columns (11,27,29,53) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../Divar.csv')


In [180]:
data.head()

,Unnamed: 0,cat2_slug,cat3_slug,city_slug,neighborhood_slug,created_at_month,user_type,description,title,rent_mode,...,property_type,regular_person_capacity,extra_person_capacity,cost_per_extra_person,rent_price_on_regular_days,rent_price_on_special_days,rent_price_at_weekends,location_latitude,location_longitude,location_radius
0,0,temporary-rent,villa,karaj,mehrshahr,2024-08-01 00:00:00,مشاور املاک,۵۰۰متر\n۲۰۰متر بنا دوبلکس\n۳خواب\nاستخر آبگرم ...,باغ ویلا اجاره روزانه استخر داخل لشکرآباد سهیلیه,NaN,...,NaN,4.0,6,350000.0,1500000.0,3.500000e+09,3500000.0,35.811684,50.936600,500.0
1,1,residential-sell,apartment-sell,tehran,gholhak,2024-05-01 00:00:00,مشاور املاک,دسترسی عالی به مترو و شریعتی \nمشاعات تمیز \nب...,۶۰ متر قلهک فول امکانات,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500.0
2,2,residential-rent,apartment-rent,tehran,tohid,2024-10-01 00:00:00,NaN,تخلیه پایان ماه,آپارتمان ۳ خوابه ۱۳۲ متر,مقطوع,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.703865,51.373459,NaN
3,3,commercial-rent,office-rent,tehran,elahiyeh,2024-06-01 00:00:00,NaN,فرشته تاپ لوکیشن\n۹۰ متر موقعیت اداری\nیک اتاق...,فرشته ۹۰ متر دفتر کار مدرن موقعیت اداری,مقطوع,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,residential-sell,apartment-sell,mashhad,emamreza,2024-05-01 00:00:00,مشاور املاک,هلدینگ ساختمانی اکبری\n\nهمراه شما هستیم برای ...,۱۱۵ متری/شمالی رو به آفتاب/اکبری,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [181]:
data.shape

(1000000, 61)

### Preprocessing and Feature engineering

#### Removing the useless Columns

In [182]:
data = data.set_index('Unnamed: 0')
data.index.name = None

In [ ]:
data = data.drop(columns=['description', 'title', 'user_type', 'rent_to_single'])

In [184]:
data.head()

,cat2_slug,cat3_slug,city_slug,neighborhood_slug,created_at_month,rent_mode,rent_value,rent_type,price_mode,price_value,...,property_type,regular_person_capacity,extra_person_capacity,cost_per_extra_person,rent_price_on_regular_days,rent_price_on_special_days,rent_price_at_weekends,location_latitude,location_longitude,location_radius
0,temporary-rent,villa,karaj,mehrshahr,2024-08-01 00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,6,350000.0,1500000.0,3.500000e+09,3500000.0,35.811684,50.936600,500.0
1,residential-sell,apartment-sell,tehran,gholhak,2024-05-01 00:00:00,NaN,NaN,NaN,مقطوع,8.500000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500.0
2,residential-rent,apartment-rent,tehran,tohid,2024-10-01 00:00:00,مقطوع,26000000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.703865,51.373459,NaN
3,commercial-rent,office-rent,tehran,elahiyeh,2024-06-01 00:00:00,مقطوع,95000000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,residential-sell,apartment-sell,mashhad,emamreza,2024-05-01 00:00:00,NaN,NaN,NaN,مقطوع,5.750000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Changing the Values of room_count column to be understandable

In [185]:
data['rooms_count'] = data['rooms_count'].replace({
    'بدون اتاق': 0,
    'یک': 1,
    'دو': 2,
    'سه': 3,
    'چهار': 4,
    'پنج یا بیشتر': 5
})

C:\Users\Abtin Rahbari\AppData\Local\Temp\ipykernel_22264\2395350444.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['rooms_count'] = data['rooms_count'].replace({


#### Combining floor count and apt in floors for column reduction

In [186]:
floors = data['total_floors_count']
floors.fillna('0', inplace=True)
#floors.unique()
floors = floors.replace({
    '30+': 31,
    'unselect': 0   
})
floors = floors.astype('int16')

upf = data['unit_per_floor']    #Units per Floor
upf.fillna('0', inplace=True)
#upf.unique()
upf = upf.replace({
    'more_than_8': 9,
    'unselect': 0   
})
upf = upf.astype('int16')
data['total_units'] = floors * upf

In [187]:
data.head()

,cat2_slug,cat3_slug,city_slug,neighborhood_slug,created_at_month,rent_mode,rent_value,rent_type,price_mode,price_value,...,regular_person_capacity,extra_person_capacity,cost_per_extra_person,rent_price_on_regular_days,rent_price_on_special_days,rent_price_at_weekends,location_latitude,location_longitude,location_radius,total_units
0,temporary-rent,villa,karaj,mehrshahr,2024-08-01 00:00:00,NaN,NaN,NaN,NaN,NaN,...,4.0,6,350000.0,1500000.0,3.500000e+09,3500000.0,35.811684,50.936600,500.0,0
1,residential-sell,apartment-sell,tehran,gholhak,2024-05-01 00:00:00,NaN,NaN,NaN,مقطوع,8.500000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500.0,0
2,residential-rent,apartment-rent,tehran,tohid,2024-10-01 00:00:00,مقطوع,26000000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,35.703865,51.373459,NaN,0
3,commercial-rent,office-rent,tehran,elahiyeh,2024-06-01 00:00:00,مقطوع,95000000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,residential-sell,apartment-sell,mashhad,emamreza,2024-05-01 00:00:00,NaN,NaN,NaN,مقطوع,5.750000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


#### Filling the water, gas, electricity, balcony, elevator, warehouse and parking columns and some other datas

In [188]:
# Correcting the has_balcony column
data.has_balcony = data.has_balcony.replace({
    'true' : True,
    'false' : False, 
    'unselect': np.nan
})
# If they had, they would have mentioned as a plus point. So we count false as default
data.has_balcony = data.has_balcony.fillna(False)

# If they had, they would have said as a plus point. So we count false as default
data.has_elevator = data.has_elevator.fillna(False)

# If they had, they would have said as a plus point. So we count false as default
data.has_warehouse = data.has_warehouse.fillna(False)

# Since it was demanded for every house to have at least one parking, default is set to true
data.has_parking = data.has_parking.fillna(True) 

# Since renovation counts as a good point, not mentioning it would count as False
data.is_rebuilt = data.is_rebuilt.fillna(False)

# it is not rational that the house wasn't mentioned that it doesn't have water, gas or electricity. So we count True as default
data.has_water = data.has_water.fillna(True)
data.has_gas = data.has_gas.fillna(True)
data.has_electricity = data.has_electricity.fillna(True)

# Since we are in Iran, we could assume any house has a single squat toilette :)
data.has_restroom = data.has_restroom.fillna('squat')

# We can mix the pool, jacuzzi and sauna and design a luxury column or even an luxury index.
# The missing values of the columns would be considered zero as default
data.has_jacuzzi = data.has_jacuzzi.fillna(False)
data.has_sauna = data.has_sauna.fillna(False)
data.has_pool = data.has_pool.fillna(False)
data.has_barbecue = data.has_barbecue.fillna(False)
data['luxury_index'] = data['has_pool'].astype('int') + data['has_sauna'].astype('int') + data['has_jacuzzi'].astype('int') + data['has_barbecue'].astype('int')

# Also shoofaj, water_cooler and package can be assumed as the default parameters for heating and cooling systems and warm water system relatively
# because it is most used (in more than have of the available data)
data.has_cooling_system = data.has_cooling_system.fillna('water_cooler')
data.has_heating_system = data.has_heating_system.fillna('shoofaj')
data.has_warm_water_provider = data.has_warm_water_provider.fillna('package')

# Most of our data is Residential so they won't have business deed by defaut
data.has_business_deed = data.has_business_deed.fillna(False)

# Most transactions only use single_page deed so we can set it as a default parameter
data.deed_type = data.deed_type.fillna('single_page')

# I assumed the 30+ floor as 31 and fill the NaN with mean+floor of the available floors
data.floor = data.floor.replace('30+', 31)
data.floor = pd.to_numeric(data.floor, errors='coerce')
data.floor = data.floor.fillna(np.floor(np.nanmean(data.floor)))

# Doing the same with room_count
data.rooms_count = data.rooms_count.fillna(np.floor(np.nanmean(data.rooms_count)))

# Binning and reforming the year column
data.construction_year = data.construction_year.replace(['قبل از ۱۳۷۰', '۱۳۷۱', '۱۳۷۲', '۱۳۷۳', '۱۳۷۴'], 1370)
data.construction_year = data.construction_year.replace(['۱۳۷۵', '۱۳۷۶', '۱۳۷۷', '۱۳۷۸', '۱۳۷۹', '۱۳۸۰', '۱۳۸۱', '۱۳۸۲', '۱۳۸۳', '۱۳۸۴'], 1380)
data.construction_year = data.construction_year.replace(['۱۳۸۵', '۱۳۸۶', '۱۳۸۷', '۱۳۸۸', '۱۳۸۹', '۱۳۹۰', '۱۳۹۱', '۱۳۹۲', '۱۳۹۳', '۱۳۹۴'], 1390)
data.construction_year = data.construction_year.replace(['۱۳۹۵', '۱۳۹۶', '۱۳۹۷', '۱۳۹۸', '۱۳۹۹', '۱۴۰۰', '۱۴۰۱', '۱۴۰۲', '۱۴۰۳'], 1400)

# Filling the missing values of construction year by city and neighborhood mean
mean_const_yr = data.groupby(['city_slug', 'neighborhood_slug']).agg({
    'construction_year': 'mean'
}).reset_index()
# Now we merge the mean data with main data
data = pd.merge(data, mean_const_yr, on=['city_slug', 'neighborhood_slug'], how='left', suffixes=('', '_mean'))
data['construction_year'] = data['construction_year'].fillna(data['construction_year_mean'])

C:\Users\Abtin Rahbari\AppData\Local\Temp\ipykernel_22264\3148537824.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.has_balcony = data.has_balcony.fillna(False)
C:\Users\Abtin Rahbari\AppData\Local\Temp\ipykernel_22264\3148537824.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.has_elevator = data.has_elevator.fillna(False)
C:\Users\Abtin Rahbari\AppData\Local\Temp\ipykernel_22264\3148537824.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.in

In [189]:
data.cat2_slug.unique()

array(['temporary-rent', 'residential-sell', 'residential-rent',
       'commercial-rent', 'commercial-sell', 'real-estate-services'],
      dtype=object)

#### Deleting the used and logically not important columns

In [ ]:
data = data.drop(columns=['unit_per_floor', 'total_floors_count', 'floor_material', 'created_at_month'])
# The security guard, barbeque column has 97 percent missing value so we can safely remove them
data = data.drop(columns=['has_security_guard', 'property_type'])
# Engineered a new feature so we can delete these three columns
data = data.drop(columns=['has_jacuzzi', 'has_sauna', 'has_pool', 'has_barbecue'])

# Building direction column is also missing 70 percent of it's data and there is no logical way to determine the missing values, so it will be deleted
data = data.drop(columns=['building_direction'])

# Regular person capacity is somehow useless because it will have data if it's a temporary rent and if it's not, it will have missing data that is 97 percent
data = data.drop(columns=['regular_person_capacity'])

# Same goes for extra person capacity (98 percent missing)
data = data.drop(columns=['extra_person_capacity'])
# since we eliminated this column we can also drop the price of extra person (99 percent missing)
data = data.drop(columns=['cost_per_extra_person'])

#### Trying the location algorithms

In [191]:
# # Extracting all Location datas into another dataset
# loc_data= data[['city_slug', 'neighborhood_slug', 'location_latitude', 'location_longitude']]

In [192]:
# loc_data.info()

In [193]:
# # Filtering the Data *with* lat and long
# df = loc_data[loc_data['location_latitude'].notnull() & loc_data['location_longitude'].notnull()].copy()

# # Separate rows with and without neighborhood labels
# df_known_neigh = df[df['neighborhood_slug'].notnull()]
# df_missing_neigh = df[df['neighborhood_slug'].isnull()]

# # Train a KNN classifier using lat and long as features
# knn = KNeighborsClassifier(n_neighbors=5)
# knn.fit(df_known_neigh[['location_latitude', 'location_longitude']], df_known_neigh['neighborhood_slug'])

# # Predict and fill the missing neighborhood values
# predicted = knn.predict(df_missing_neigh[['location_latitude', 'location_longitude']])

# # Putting it back into loc_data
# loc_data.loc[df_missing_neigh.index, 'neighborhood_slug'] = predicted

In [194]:
# loc_data.info()

In [195]:
# # Designing a function to fill the neighborhood column for rows that only have city data by replacing the NaN data with most frequent neighborhood
# def fill_with_frequent(group):
#     if group.mode().empty:
#         # Doing this to check if we have some cities that do not have any neighborhoods
#         return
#     else:
#         return group.fillna(group.mode().iloc[0])

# loc_data['neighborhood_slug'] = loc_data.groupby('city_slug')['neighborhood_slug'].transform(fill_with_frequent)

In [196]:
# loc_data[loc_data['city_slug'].isna()]

This data has no location Data therefore can't be used in the secondary part of the question. Because of that we delete it

In [197]:
# data = data.drop(loc_data[loc_data['city_slug'].isna()].index)
# loc_data = loc_data.drop(loc_data[loc_data['city_slug'].isna()].index)
# loc_data.info()

Now we fill the lat and long data based on City and Neighborhoods

In [198]:
# # Calculating the center of each neighborhood for lat and long filling 
# centroids = loc_data.groupby(['city_slug', 'neighborhood_slug']).agg({
#     'location_latitude': 'mean',
#     'location_longitude': 'mean'
# }).reset_index()

# # Now we merge the centres with main data
# full_loc = pd.merge(loc_data, centroids, on=['city_slug', 'neighborhood_slug'], how='left', suffixes=('', '_centers'))
# full_loc['location_latitude'] = full_loc['location_latitude'].fillna(full_loc['location_latitude_centers'])
# full_loc['location_longitude'] = full_loc['location_longitude'].fillna(full_loc['location_longitude_centers'])

In [199]:
# full_loc = full_loc.drop(columns=['location_latitude_centers', 'location_longitude_centers'])
# full_loc.info()

In [200]:
# # We still have two neighborhoods that have no location data. Because the count is so little (4 samples) we can delete them
# full_loc = full_loc.drop(full_loc[full_loc['location_latitude'].isna()].index)
# data = data.drop(full_loc[full_loc['location_latitude'].isna()].index)

#### Now that we now that the algorithm works, we execute it on the main dataset

In [201]:
# Designing a function to fill the neighborhood column for rows that only have city data by replacing the NaN data with most frequent neighborhood
def fill_with_frequent(group):
    if group.mode().empty:
        # Doing this to check if we have some cities that do not have any neighborhoods
        return
    else:
        return group.fillna(group.mode().iloc[0])

# Filtering the Data *with* lat and long
df = data[data['location_latitude'].notnull() & data['location_longitude'].notnull()].copy()

# Separate rows with and without neighborhood labels
df_known_neigh = df[df['neighborhood_slug'].notnull()]
df_missing_neigh = df[df['neighborhood_slug'].isnull()]

# Train a KNN classifier using lat and long as features
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(df_known_neigh[['location_latitude', 'location_longitude']], df_known_neigh['neighborhood_slug'])

# Predict and fill the missing neighborhood values
predicted = knn.predict(df_missing_neigh[['location_latitude', 'location_longitude']])

# Putting it back into loc_data
data.loc[df_missing_neigh.index, 'neighborhood_slug'] = predicted

data['neighborhood_slug'] = data.groupby('city_slug')['neighborhood_slug'].transform(fill_with_frequent)
data = data.drop(data[data['city_slug'].isna()].index)      # Getting rid of data with no location

# Calculating the center of each neighborhood for lat and long filling 
centers = data.groupby(['city_slug', 'neighborhood_slug']).agg({
    'location_latitude': 'mean',
    'location_longitude': 'mean'
}).reset_index()

# Now we merge the centres with main data
data = pd.merge(data, centers, on=['city_slug', 'neighborhood_slug'], how='left', suffixes=('', '_centers'))
data['location_latitude'] = data['location_latitude'].fillna(data['location_latitude_centers'])
data['location_longitude'] = data['location_longitude'].fillna(data['location_longitude_centers'])

data = data.drop(columns=['location_latitude_centers', 'location_longitude_centers'])
data = data.drop(data[data['location_latitude'].isna()].index)

#### Completing the cat3 column based on the cat2

In [202]:
data['cat3_slug'] = data.groupby('cat2_slug')['cat3_slug'].transform(fill_with_frequent)

In [203]:
data.cat2_slug.unique()

array(['temporary-rent', 'residential-sell', 'residential-rent',
       'commercial-rent', 'commercial-sell', 'real-estate-services'],
      dtype=object)

#### Now Let's work on prices

In [221]:
# When we have sell -> We go to price value
# When we have non-temporary rent -> we go to transformed credit and rent and make it a full credit
# When we have temporary rent -> we convert the (special + weekend + normal) daily rent into full credit

### For temporary rent
temp_rent = data[data['cat2_slug'] == 'temporary-rent']
mean_narrow = temp_rent.groupby(['city_slug', 'neighborhood_slug']).agg({
    'building_size': 'mean',
    'rent_price_on_regular_days': 'mean',
    'rent_price_on_special_days': 'mean',
    'rent_price_at_weekends': 'mean'
}).reset_index()
temp_rent = pd.merge(temp_rent, mean_narrow, on=['city_slug', 'neighborhood_slug'], how='left', suffixes=('', '_mean'))
temp_rent['rent_price_on_regular_days_mean'] = temp_rent['rent_price_on_regular_days_mean'].fillna(0)
temp_rent['rent_price_on_special_days_mean'] = temp_rent['rent_price_on_special_days_mean'].fillna(0)
temp_rent['rent_price_at_weekends_mean'] = temp_rent['rent_price_at_weekends_mean'].fillna(0)
temp_rent['rent_price_on_regular_days'] = temp_rent['rent_price_on_regular_days'].fillna(temp_rent['rent_price_on_regular_days_mean']*temp_rent['building_size']/temp_rent['building_size_mean'])
temp_rent['rent_price_on_special_days'] = temp_rent['rent_price_on_special_days'].fillna(temp_rent['rent_price_on_special_days_mean']*temp_rent['building_size']/temp_rent['building_size_mean'])
temp_rent['rent_price_at_weekends'] = temp_rent['rent_price_at_weekends'].fillna(temp_rent['rent_price_at_weekends_mean']*temp_rent['building_size']/temp_rent['building_size_mean'])
temp_rent = temp_rent.drop(columns=['rent_price_on_regular_days_mean', 'rent_price_on_special_days_mean', 'rent_price_at_weekends_mean'])

temp_rent = temp_rent.drop(temp_rent[(data['cat2_slug'] == 'temporary-rent') & ((temp_rent['rent_price_at_weekends'].isna()) & (temp_rent['rent_price_on_regular_days'].isna()) & (temp_rent['rent_price_on_special_days'].isna()))].index)

# Calculating final credit by the assumptions that we have 104 weekends, 13 days for nowruz holidays
# and average of two days per month for holidays and the rest are regular days.
temp_rent['final_credit'] = (((52*2) * temp_rent['rent_price_at_weekends']) + ((13 + (2*11))*temp_rent['rent_price_on_special_days']) + ((365 - ((52*2) + (13 + (2*11))))*temp_rent['rent_price_on_regular_days'])) * 1000000 / 30000

C:\Users\Abtin Rahbari\AppData\Local\Temp\ipykernel_22264\2641319856.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp_rent = temp_rent.drop(temp_rent[(data['cat2_slug'] == 'temporary-rent') & ((temp_rent['rent_price_at_weekends'].isna()) & (temp_rent['rent_price_on_regular_days'].isna()) & (temp_rent['rent_price_on_special_days'].isna()))].index)


In [222]:
temp_rent

,cat2_slug,cat3_slug,city_slug,neighborhood_slug,rent_mode,rent_value,rent_type,price_mode,price_value,credit_mode,...,rent_price_on_special_days,rent_price_at_weekends,location_latitude,location_longitude,location_radius,total_units,luxury_index,construction_year_mean,building_size_mean,final_credit
0,temporary-rent,villa,karaj,mehrshahr,NaN,NaN,NaN,NaN,NaN,NaN,...,3.500000e+09,3.500000e+06,35.811684,50.936600,500.0,0,0,1396.878267,1082.105263,4.106767e+12
1,temporary-rent,villa,mashhad,emamreza,NaN,NaN,NaN,NaN,NaN,NaN,...,7.332284e+05,2.500000e+06,36.266147,59.580842,NaN,0,0,1395.413070,1107.458453,2.082210e+10
2,temporary-rent,villa,khoram-abad,abr,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000e+00,0.000000e+00,36.639404,50.756630,NaN,0,1,NaN,133.125000,2.260000e+10
3,temporary-rent,villa,shahin-dej,arian-town,NaN,NaN,NaN,NaN,NaN,NaN,...,5.800000e+07,5.800000e+07,36.678196,46.592247,NaN,0,0,NaN,226716.000000,7.056667e+11
4,temporary-rent,suite-apartment,kish,azar-isfahan,NaN,NaN,NaN,NaN,NaN,NaN,...,2.800000e+06,2.200000e+06,26.521957,53.982102,NaN,0,0,NaN,868.102190,2.445333e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29898,temporary-rent,villa,shiraz,goldashthafez,NaN,NaN,NaN,NaN,NaN,NaN,...,4.000000e+06,3.000000e+06,29.730723,52.428223,NaN,0,0,1397.183544,856.500000,3.390000e+10
29899,temporary-rent,suite-apartment,qom,haram,NaN,NaN,NaN,NaN,NaN,NaN,...,4.007637e+04,4.261455e+04,34.641823,50.878948,NaN,0,0,1388.571429,710.681159,5.984921e+08
29900,temporary-rent,villa,dezful,amaniyeh-ahvaz,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000e+00,2.500000e+06,31.542261,48.313751,NaN,0,0,NaN,1000.000000,1.996667e+10
29901,temporary-rent,suite-apartment,marivan,khorramdasht,NaN,NaN,NaN,NaN,NaN,NaN,...,6.687157e+05,5.440036e+05,35.519154,46.187916,NaN,0,0,NaN,177.147826,1.350037e+17


In [217]:
### For rents
rent_data = data[(data['cat2_slug'] == 'residential-rent') | (data['cat2_slug'] == 'commercial-rent')]
rent_data[rent_data.transformable_credit.isna()].shape[0] / rent_data[rent_data.transformable_credit.notna()].shape[0]

0.0029538405607752696

Because we have a lot more data which they have the transformable_credit data, we could remove the ones without that data

In [218]:
rent_data = rent_data[rent_data.transformable_credit.notna()]

Now we calculate the full credit for each one

### Implementing Kmeans

In [ ]:
kmean = KMeans(10, random_state=42)
clusters = kmean.fit_predict(data)